# 예측 : 저장된 비디오의 경우
* 이전의 MobileNet, 훈련된 LSTM 모델을 사용하여 저장된 비디오 파일(.avi, .mp4)의 폭력을 예측하여 저장

# Imports

In [1]:
import cv2 # openCV 4.5.1
import numpy as np # numpy 배열
import os # 파일 및 폴더의 경로 지정을 위한 모듈
import tensorflow as tf # 텐서플로우
from tensorflow import keras # 케라스
import time #프로세스 소요시간 표시 목적

from skimage.io import imread #이미지 보이기
from skimage.transform import resize # 이미지 리사이즈

from PIL import Image, ImageFont, ImageDraw #자막 투입 목적
from io import BytesIO

from collections import deque #비디오 영상에 텍스트 씌워 저장하기에 사용

# 모델 불러오기
* 01~03 의 과정을 거쳐 저장된 모델을 불러오기

## 이미지를 투입할 베이스 모델(MobileNet)

In [2]:
base_model=keras.applications.mobilenet.MobileNet(input_shape=(160, 160, 3),
                                                  include_top=False,
                                                  weights='imagenet', classes=2)

## 과거 훈련시킨 LSTM 모델(.h5)불러오기

In [3]:
model=keras.models.load_model('210508_vc_MobileNet_model_epoch100.h5')

# 필요 함수 정의

## 함수 : 비디오 파일 열기 > 스케일링, 리사이징
* 비디오 파일을 불러와 각 프레임들을 넘파이 배열로 변환하여 반환

In [4]:
def video_reader(cv2, filename):
    """비디오 파일 1개를 불러와 각 장면과 프레임을 읽고,
    (frame, 160, 160, 3) 크기의 배열로 리사이징한 뒤,
    해당 프레임들을 반환"""
    frames=np.zeros((30, 160, 160, 3), dtype=np.float)
    #> (비디오파일 개수, 프레임 개수, 사이즈, 사이즈, RGB)
    i=0
    print(frames.shape)
    vc=cv2.VideoCapture(filename) # 비디오 파일로부터 프레임별 이미지 읽어오기
    
    if vc.isOpened(): #비디오 파일이 열려있으면
        rval, frame=vc.read() #영상의 프레임, 값 추출
    else:
        rval=False
    
    frm=resize(frame,(160, 160, 3))
    frm=np.expand_dims(frm, axis=0)
    
    if(np.max(frm)>1):
        frm=frm/255.0 #프레임의 RGB 값을 스케일링 합니다
    frames[i][:]=frm
    i+=1
    print('Reading Video')
    
    while i<30:
        rval, frame=vc.read()
        frm=resize(frame, (160, 160, 3)) #영상을 ()
        frm=np.expand_dims(frm, axis=0)
        if(np.max(frm)>1):
            frm=frm/255.0
        frames[i][:]=frm
        i+=1
        
    return frames #> 비디오를 읽어 (30, 160, 160, 3) 배열을 리턴한다.

## 함수 : MobileNet로 프레임별 특성 추출
* base_model에 각 프레임을 투입하여, 특성 배열을 추출
* 추출한 프레임 이미지별 특성 배열을 훈련된 LSTM모델에 투입할 수 있도록 1차원 배열로 변환함

In [5]:
def create_pred_imgarr(base_model, video_frm_ar):
    """함수 video_reader()사용 결과물을 투입
    base_model(MobileNet)이 프레임별로 특성을 추출해 준 뒤,
    추출된 프레임 이미지별 특성 배열을
    훈련된 LSTM모델에 투입가능한 형태(1차원 배열)로 변환해 줌"""
    # video_frm_ar을 차원 확장 : (30, 160, 160, 3) 투입
    #data=[]
    #data.append(video_frm_ar)
    #video_frm_ar_dim=np.array(data)
    video_frm_ar_dim=np.zeros((1, 30, 160, 160, 3), dtype=np.float)
    video_frm_ar_dim[0][:][:]=video_frm_ar #> (1, 30, 160, 160, 3)
     
    # MobileNet으로 각 프레임 이미지로부터 특성 배열 추출
    pred_imgarr=base_model.predict(video_frm_ar)
    # 추출된 특성 배열을 1차원으로 변환: (1, 프레임 수, 25600)
    pred_imgarr=pred_imgarr.reshape(1, pred_imgarr.shape[0], 5*5*1024)
    
    return pred_imgarr #> ex : (1, 30, 25600)

## 함수: 폭력여부 판별
* 1차원으로 변환된 프레임별 특성 배열을 투입 훈련된 LSTM모델에 투입
* 모델로부터 폭력인지/비폭력인지 여부 판별

In [6]:
def pred_fight(model, pred_imgarr, acuracy=0.9):
    """예측값이 지정 acuracy이상이면 'True'(폭력)를,
    그 미만이면 False(비폭력) 반환.
    
    ::model:: 훈련된 LSTM 모델(불러오기)
    ::pred_imgarr:: (1, 30, 25600) 형태, 1개 영상의 추출된 특성
    ::accuracy:: 0.9가 디폴트값. 임의의 값 지정 가능"""

    pred_test=model.predict(pred_imgarr)
    #> [0,1](폭력) 또는 [1,0](비폭력)으로 설정되었음에 유의!
    
    if pred_test[0][1] >= acuracy:
        return True, pred_test[0][1] #> True, 폭력일 확률
    
    else:
        return False, pred_test[0][1] #> False, 폭력일 확률

## 함수 기능 검사

In [7]:
video_file='Fight_itwill_210506_01.mp4'

In [8]:
video_frm_ar=video_reader(cv2, video_file)

(30, 160, 160, 3)
Reading Video


In [9]:
pred_imgarr=create_pred_imgarr(base_model, video_frm_ar)
pred_imgarr.shape

(1, 30, 25600)

In [10]:
preds=pred_fight(model, pred_imgarr, 0.9)
preds
#> 폭력. 폭력일 확률

(True, 0.99869007)

# 비디오 파일 투입, 폭력여부 판별
* 비디오 파일(.mp4) 1개 투입 > 폭력 여부(True/False) 및 폭력일 확률 출력
* 정의한 3개의 함수를 모두 사용하여 결과 도출

In [11]:
def main_fight(video):
    """1개의 영상을 넣고 정의한 3개의 함수를 모두 사용하여 결과 도출
    video_reader() : openCV를 사용해 비디오 불러옴
    create_pred_imgarr() : MobileNet으로 이미지별 특성값 저장
    pred_fight() : 훈련된 LSTM모델로 폭력여부 예측"""
    
    # video_reader()함수로 비디오 읽기
    video_frm_ar=video_reader(cv2, video) 
    # 읽어온 비디오를 VGG19로 특성을 추출하고 알맞게 형변환
    pred_imgarr=create_pred_imgarr(base_model, video_frm_ar)
    #datav=np.zeros((1, 30, 160, 160, 3), dtype=np.float)
    #datav[0][:][:]=vid
    
    millis=int(round(time.time()*1000)) #> 프로세스 시작시각
    
    # LSTM 모델(model)에 데이터를 투입, 65%이상이면 폭력으로 표시
    f, precent=pred_fight(model, pred_imgarr, acuracy=0.65)
    
    millis2=int(round(time.time()*1000)) #> 프로세스 종료시각
    
    res_fight={'Violence': f, #> 폭력(True), 비폭력(False)
               'Violence Estimation': str(precent), # 폭력확률
               'Processing Time' : str(millis2-millis)} #> 소요시간
    
    return res_fight

In [12]:
# 영상을 넣은 경우
video_file='Fight_itwill_210506_01.mp4'
main_fight(video_file)

(30, 160, 160, 3)
Reading Video


{'Violence': True,
 'Violence Estimation': '0.99869007',
 'Processing Time': '148'}

# 비디오 영상에 텍스트 씌워 저장하기
* collections 모듈의 deque(데크)double-ended queue 의 줄임말로, 앞과 뒤에서 즉, 양방향에서 데이터를 처리할 수 있는 queue형 자료구조입니다.
* 파이썬에서의 리스트와 다루는 법이 유사한 편입니다.
* deque 참고 : https://excelsior-cjh.tistory.com/96

## 투입 비디오 설정(input)
* **주의!** 투입 비디오 프레임 수는 반드시 **30**이어야 합니다. 29.97 안됩니다

In [23]:
input_path='Fight_itwill_210506_05.mp4' # 투입할 동영상 파일 이름 및 경로
output_path=f'{input_path}+output.mp4' #폭력 여부 감별 후 자막을 씌워 저장할 파일 이름 및 경로

In [24]:
#. ----- 비디오 스트리밍을 불러오기 & 초기설정 -----
vc=cv2.VideoCapture(input_path)
fps=vc.get(cv2.CAP_PROP_FPS) # input_path의 초당 프레임 수 인식. fps=30.0
print(f'fps : {fps}')

writer=None
(W, H)=(None, None)
i=0 # 비디오 초 번호. While loop를 돌아가는 회차
Q=deque(maxlen=128) 

video_frm_ar=np.zeros((1, int(fps), 160, 160, 3), dtype=np.float) #frames
frame_counter=0 # 1초당 프레임 번호. 1~30
frame_list=[] 
preds=None
maxprob=None

#. While loop : 스트리밍이 끝날 때까지 프레임 추출 반복문 시작
# ----- 스트리밍 동영상들을 (30, 160, 160, 3)으로 저장하기 시작 -----
while True: 
    frame_counter+=1
    grabbed, frm=vc.read() # 비디오를 1개 프레임씩 읽는다.
    #> grabbed=True, frm=프레임별 넘파이 배열. (240, 320, 3)
    
    if not grabbed: # 프레임이 안 잡힐 경우
        print('프레임이 없습니다. 스트리밍을 종료합니다.')
        break
            
    #if fps!=30: #비디오 fps가 30이 아니면 루프를 돌지 않기로 한다.
        #print('비디오의 초당 프레임이 30이 아닙니다. fps=30으로 맞춰주세요.')
        #break
        
    if W is None or H is None: # 프레임 이미지 폭(W), 높이(H)를 동영상에서
        (H, W)=frm.shape[:2]
            
    output=frm.copy() #비디오 프레임을 그대로 복사. 저장/출력할 .mp4 파일로
    
    frame=resize(frm, (160, 160, 3)) #> input 배열을 (160, 160, 3)으로 변환
    frame_list.append(frame) #각 프레임 배열 (160, 160, 3)이 append 된다.
        
    if frame_counter>=fps: #프레임 카운터가 30이 된 순간. len(frame_list)==30이 된 순간.
        #. ----- 1초=30프레임마다 묶어서 예측(.predict) -----
        #. ----- 1초 동안 (1, 30, 160, 160, 3) 배열을 만들어 모델에 투입 ---
        #. ----- 예측 결과(1초)를 output에 씌워 준다. -----
        # 그러기 위해서는 30개씩 append한 리스트를 넘파이화 -> 예측 -> 리스트 초기화 과정이 필요
        frame_ar=np.array(frame_list, dtype=np.float16) #> 30개의 원소가 든 리스트를 변환. (30, 160, 160, 3)
        frame_list=[] #30프레임이 채워질 때마다 넘파이 배열로 변환을 마친 프레임 리스트는 초기화 해 준다.
            
        if(np.max(frame_ar)>1): # 넘파이 배열의 RGB 값을 스케일링
            frame_ar=frame_ar/255.0
            
        #video_frm_ar[i][:]=frame_ar #> (i, fps, 160, 160, 3). i번째 1초짜리 영상(30프레임) 배열 파일이 된다.
        #print(video_frm_ar.shape)
        
        #MobileNet로 초당 프레임 이미지 배열로부터 특성 추출 : (1*30, 5, 5, 1024)
        pred_imgarr=base_model.predict(frame_ar) #> (30, 5, 5, 1024)
        # 추출된 특성 배열들을 1차원으로 변환 : (1, 30, 5*5*1024)
        pred_imgarr_dim=pred_imgarr.reshape(1, pred_imgarr.shape[0], 5*5*1024)#> (1, 30, 25600)
        # 각 프레임 폭력 여부 예측값을 0에 저장
        preds=model.predict(pred_imgarr_dim) #> (True, 0.99) : (폭력여부, 폭력확률)
        print(f'preds:{preds}')
        Q.append(preds) #> Deque Q에 리스트처럼 예측값을 추가함
    
        # 지난 5초간의 폭력 확률 평균을 result로 한다.
        if i<5:
            results=np.array(Q)[:i].mean(axis=0)
        else:
            results=np.array(Q)[(i-5):i].mean(axis=0)
        #results=np.array(Q).mean(axis=0)
        print(f'Results = {results}') #> ex : (0.6, 0.650)
            
        #예측 결과에서 최대폭력확률값
        maxprob=np.max(results) #> 가장 높은 값을 선택함
        print(f'Maximum Probability : {maxprob}')
        print('')
            
        rest=1-maxprob # 폭력이 아닐 확률
        diff=maxprob-rest # 폭력일 확률과 폭력이 아닐 확률의 차이
        th=100
            
        if diff>0.80: # 폭력일 확률과 아닐 확률의 차이가 0.8 이상이면
            th=diff #근거가?
        
        frame_counter=0 #> 1초(30프레임)가 경과했으므로 frame_counter=0으로 리셋
        i+=1 #> 1초 경과 의미
        
        # frame_counter==30이 되면 0으로 돌아가 위 루프를 반복해 준다.
                
    # ----- output에 씌울 자막 설정하기 -----
    # 30프레임(1초)마다 갱신된 값이 output에 씌워지게 된다
    font1=ImageFont.truetype('fonts/Raleway-ExtraBold.ttf', int(0.05*W))
    font2=ImageFont.truetype('fonts/Raleway-ExtraBold.ttf', int(0.1*W))
    
    if preds is not None and maxprob is not None: # 예측값이 발생한 후부터
        if (preds[0][1])<th : #> 폭력일 확률이 th보다 작으면 정상
            text1_1='Normal'
            text1_2='{:.2f}%'.format(100-(maxprob*100))
            #cv2.putText(output, text1_1, (int(0.025*W), int(0.1*H)),font, fontScale, (0, 255, 0), 2)
            #cv2.putText(output, text1_2, (int(0.025*W), int(0.2*H)),font, fontScale, (0, 255, 0), 2)
            img_pil=Image.fromarray(output)
            draw=ImageDraw.Draw(img_pil)
            draw.text((int(0.025*W), int(0.025*H)), text1_1, font=font1, fill=(0,255,0,0))
            draw.text((int(0.025*W), int(0.095*H)), text1_2, font=font2, fill=(0,255,0,0))
            output=np.array(img_pil)
            # cv2.putText(이미지파일, 출력문자, 시작위치좌표(좌측하단), 폰트, 폰트크기, 폰트색상, 폰트두께)
                
        else : #> 폭력일 확률이 th보다 크면 폭력 취급
            text2_1='Violence Alert!'
            text2_2='{:.2f}%'.format(maxprob*100)
            #cv2.putText(output, text2_1, (int(0.025*W), int(0.1*H)),font, fontScale, (0, 0, 255), 2)
            #cv2.putText(output, text2_2, (int(0.025*W), int(0.2*H)),font, fontScale, (0, 0, 255), 2) 
            img_pil=Image.fromarray(output)
            draw=ImageDraw.Draw(img_pil)
            draw.text((int(0.025*W), int(0.025*H)), text2_1, font=font1, fill=(0,0,255,0))
            draw.text((int(0.025*W), int(0.095*H)), text2_2, font=font2, fill=(0,0,255,0))
            output=np.array(img_pil)
        
    # 자막이 씌워진 동영상을 writer로 저장함
    if writer is None:
        fourcc=cv2.VideoWriter_fourcc(*'DIVX')
        writer=cv2.VideoWriter(output_path, fourcc, 30, (W, H), True)
            
    # 아웃풋을 새창으로 열어 보여주기
    cv2.imshow('This is output', output)
    writer.write(output) #output_path로 output 객체를 저장함
        
    key=cv2.waitKey(round(1000/fps)) # 프레임-다음 프레임 사이 간격
    if key==27: # esc 키를 누르면 루프로부터 벗어나고 output 파일이 저장됨
        print('ESC키를 눌렀습니다. 녹화를 종료합니다.')
        break
    
print('종료 처리되었습니다. 메모리를 해제합니다.')
writer.release()
vc.release()
cv2.destroyAllWindows()

fps : 30.061038063436285
preds:[[2.2413046e-04 9.9977583e-01]]
Results = [[nan nan]]
Maximum Probability : nan



<ipython-input-24-bf36c8d9aed1>:65: RuntimeWarning: Mean of empty slice.
  results=np.array(Q)[:i].mean(axis=0)


preds:[[0.00478598 0.995214  ]]
Results = [[2.2413046e-04 9.9977583e-01]]
Maximum Probability : 0.9997758269309998

preds:[[7.171974e-04 9.992828e-01]]
Results = [[0.00250505 0.99749494]]
Maximum Probability : 0.9974949359893799

preds:[[1.7574173e-04 9.9982435e-01]]
Results = [[0.0019091 0.9980909]]
Maximum Probability : 0.998090922832489

preds:[[5.1806617e-04 9.9948198e-01]]
Results = [[0.00147576 0.99852425]]
Maximum Probability : 0.9985242486000061

preds:[[0.00106501 0.9989349 ]]
Results = [[0.00128422 0.99871576]]
Maximum Probability : 0.9987157583236694

preds:[[0.0015095  0.99849045]]
Results = [[0.0014524  0.99854755]]
Maximum Probability : 0.9985475540161133

preds:[[4.4945168e-04 9.9955052e-01]]
Results = [[7.9710456e-04 9.9920291e-01]]
Maximum Probability : 0.9992029070854187

preds:[[7.523823e-05 9.999248e-01]]
Results = [[7.4355537e-04 9.9925643e-01]]
Maximum Probability : 0.999256432056427

preds:[[5.567938e-05 9.999443e-01]]
Results = [[7.2345464e-04 9.9927652e-01]]
Ma